In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
import xarray as xr
hv.notebook_extension()
%opts Scatter (color='b') Path [show_legend=True aspect='square'] Layout [show_title=False] Image [aspect='square']
pd.set_option('display.max_columns', None)

def drop_constant_columns(df):
    """Taken from http://stackoverflow.com/a/20210048/3447047"""
    return df.loc[:, (df != df.ix[0]).any()]

def hm_of_df(fname, ymax=None):
    df = pd.read_hdf(fname)
    df['(leads, L_sc)'] = [str(t) for t in zip(df.with_leads, df.L_sc)]
    df2 = df.drop(['with_leads', 'L_sc', 'phase_c'], axis=1)  # because of line above
    df2 = df2.drop(['currents', 'phases', 'git_hash'], axis=1)  # because not hashable
    df2 = drop_constant_columns(df2)  # because collumns contain constants
    ds = hv.Dataset(df2)
    hm = ds.to.curve(kdims='B_x', vdims='current_c', dynamic=True)
    hm = hm.redim(current_c=dict(range=(0, ymax)))
    return hm

# Disorder with wire length $L=640$ nm

In [ ]:
# from glob import glob
# files = glob('tmp/I_c_B_x_disorder*')
# df = pd.concat([pd.read_hdf(f) for f in files])
# df = df.sort_values(['salt', 'disorder', 'orbital', 'g', 'alpha', 'mu', 'B_x'])
# df = df.reset_index(drop=True)
# df.to_hdf('data/current_as_function_of_B_x_with_disorder_with_combinations_of_effects.hdf', 'all_data')

hm_of_df('data/current_as_function_of_B_x_with_disorder_with_combinations_of_effects.hdf', 25)

# Disorder with wire length $L=1000$ nm

In [ ]:
# from glob import glob
# files = glob('tmp/I_c_B_x_disorder*')
# df = pd.concat([pd.read_hdf(f) for f in files])
# df = df.sort_values(['salt', 'disorder', 'orbital', 'g', 'alpha', 'mu', 'B_x'])
# df = df.reset_index(drop=True)
# df.to_hdf('data/current_as_function_of_B_x_with_disorder_with_combinations_of_effects_new.hdf', 'all_data')

hm_of_df('data/current_as_function_of_B_x_with_disorder_with_combinations_of_effects_new.hdf', 19)

# All combinations of effects
### Different panels are to show that the vector potential in the SC doesn't effect the current much

In [ ]:
# For a square
hm_of_df('data/all_data_square_A_in_SM.hdf', 140)

In [ ]:
# For a square wire
hm_of_df('data/all_data_square_A_in_SM_and_SC.hdf', 140)

In [ ]:
ymax = 90
(hm_of_df('data/all_data_circular_A_in_SM_incorrect_vector_potential.hdf', ymax) +
 hm_of_df('data/all_data_circular_A_in_SM.hdf', ymax) +
 hm_of_df('data/all_data_circular_A_in_SM_and_SC.hdf', ymax))

# Gate without disorder

In [ ]:
%%opts GridImage [aspect='square']
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x__A_in_SM_and_SC.hdf')
da = df.set_index(['gate_size', 'B_x', 'Vs', 'disorder']).to_xarray()
ds = hv.Dataset(da)
ds.to(hv.GridImage, kdims=['B_x', 'Vs'], vdims=['current_c'])

In [ ]:
%%opts GridImage [aspect='square']
# incorrect vector potential
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x__A_in_SM_incorrect_vector_potential.hdf')
da = df.set_index(['gate_size', 'B_x', 'Vs', 'disorder']).to_xarray()
ds = hv.Dataset(da)
ds.to(hv.GridImage, kdims=['B_x', 'Vs'], vdims=['current_c'])

# gates with disorder
* mu: 50 meV, disorder: 20 meV, mfp: 3147 nm
* mu: 50 meV, disorder: 50 meV, mfp: 656 nm
* mu: 50 meV, disorder: 70 meV, mfp: 357 nm

In [ ]:
%%opts GridImage [aspect='square'] {+framewise}

# from glob import glob
# files = glob('tmp/current_as_function_of_gate_and_B_x__gate_size_*')
# df = pd.concat([pd.read_hdf(f) for f in files])
# df = df.sort_values(['gate_size', 'B_x', 'V', 'disorder'])
# df = df.reset_index(drop=True)
# df.to_hdf('data/current_as_function_of_gate_and_B_x_with_disorder.hdf', 'all_data')

df = pd.read_hdf('data/current_as_function_of_gate_and_B_x_with_disorder_combined.hdf')
da = df.set_index(['gate_size', 'B_x', 'V', 'disorder']).to_xarray()
ds = hv.Dataset(da)
ds.to(hv.GridImage, kdims=['B_x', 'V'], vdims=['current_c'])

# Rotation of field

In [ ]:
%%opts Path [aspect='square']
to_deg = lambda col: np.round(col * 180 / np.pi, 1).astype(int)

df = pd.read_hdf('data/rotation_of_field__L_640_nm__infinite_leads.hdf')
df['theta'] = df['theta'].apply(to_deg)
df2 = df.drop(['currents', 'phases', 'git_hash'], axis=1)  # because not hashable
ds = hv.Dataset(df2)
current = ds.to.curve(kdims='B', vdims='current_c', groupby='theta').redim(current_c=dict(range=(0, None)))
phase = ds.to.curve(kdims='B', vdims='phase_c', groupby='theta')
current + phase